In [31]:
import random
from dataclasses import dataclass, field
from typing import List, Dict, Any

import numpy as np
import os
from tampura.environment import TampuraEnv
from tampura.spec import ProblemSpec
from tampura.structs import (
    AbstractBelief,
    ActionSchema,
    StreamSchema,
    AliasStore,
    Belief,
    NoOp,
    Predicate,
    State,
    effect_from_execute_fn,
    Observation
)
import logging 
from tampura.symbolic import OBJ, Or, Atom, ForAll, Exists, OneOf, And, Not, Eq, negate
from tampura.policies.tampura_policy import TampuraPolicy
from tampura.config.config import get_default_config, setup_logger


PICK_SUCCESS = 1.0
DROP_SUCCESS = 1.0

OBJECTS = {f"{OBJ}o1":{"region":1,"size":10},f"{OBJ}o2":{"region":2,"size":4}}
ROBOT = {"region":5,"gripper_region":5}
DICE = {"region": 2}

THRESH = 0.5

In [32]:
# Observation space


@dataclass 
class ObjectObservation(Observation):
    
    at: Dict[str,str]=field(default_factory=lambda: {})
    atRob: str=field(default_factory=lambda: "")
    inspected: List[str]=field(default_factory=lambda: [])
    inspected_configurations: List[str]=field(default_factory=lambda: [])
    found: List[str]=field(default_factory=lambda: [])
    

# Belief space
class ObjectsBelief(Belief):
    def __init__(self, at={}, atRob="", inspected=[], inspected_configurations=[], found=[]):
        
        self.at=at
        self.atRob=atRob
        self.inspected=inspected  
        self.inspected_configurations=inspected_configurations
        self.found=found
  

    def update(self, a, o, s):
        
        at=o.at
        atRob=o.atRob
        inspected=o.inspected
        inspected_configurations=o.inspected_configurations
        found=o.found

        
        return ObjectsBelief(at=at,atRob=atRob,inspected=inspected,inspected_configurations=inspected_configurations,found=found) 

    def abstract(self, store: AliasStore):
        
        atoms = []
        
        if self.atRob != "":
            atoms.append(Atom("atRob",[self.atRob]))
        
        for o in self.at.keys():
            atoms.append(Atom("at",[o,self.at[o]]))
        
        for o in self.inspected:
            atoms.append(Atom("inspected", [o]))
            
        for q in self.inspected_configurations:
            atoms.append(Atom("inspected_configuration",[q]))
            
        for o in self.found:
            atoms.append(Atom("found", [o]))       
            
            
        return AbstractBelief(atoms)
    

    def vectorize(self):
        pass
    
# Sample function for stream schema
def sample_robot_location(input_sym, store):
    
    return ROBOT["region"]


def sample_gripper(input_sym, store):
    
    return ROBOT["gripper_region"]

def sample_location(input_sym, store):
    
    o = input_sym[0]

    
    r = OBJECTS[o]["region"]

    
    return r
    
def sample_grasp(input_sym, store):
    
    g = random.random()
    
    # TODO: make more sophisticated
    
    return g

    
# action simulators
def look_execute_fn(a, b, s, store):
    
    q = a.args[0]
    
    # forward kinematics
    # TODO: finish!!!
    at=b.at.copy()
    atRob=b.atRob
    inspected=b.inspected.copy()
    inspected_configurations=b.inspected_configurations.copy()
    found=b.found.copy()
        
    if store.get(q) == DICE["region"]:
        
        found=list(set(found+["dice"]))
        
    inspected_configurations=list(set(inspected_configurations+[q]))
    
    return State(), ObjectObservation(at=at,atRob=atRob,inspected=inspected,inspected_configurations=inspected_configurations,found=found)
    
    
    
    
    
def move_execute_fn(a, b, s, store):
    
    q = a.args[0]
    
    at=b.at.copy()
    atRob=b.atRob
    inspected=b.inspected.copy()
    inspected_configurations=b.inspected_configurations.copy()
    found=b.found.copy()
   
    # effects
    atRob=q
    
    return State(), ObjectObservation(at=at,atRob=atRob,inspected=inspected,inspected_configurations=inspected_configurations,found=found)
    

def pick_execute_fn(a, b, s, store):
    
    o = a.args[0]
    g = a.args[1]
    
    at=b.at.copy()
    atRob=b.atRob
    inspected=b.inspected.copy()
    inspected_configurations=b.inspected_configurations.copy()
    found=b.found.copy()

    
    # verify_effects
    if random.random() < PICK_SUCCESS:
        at[o]="robot_gripper"
    
    return State(), ObjectObservation(at=at,atRob=atRob,inspected=inspected,inspected_configurations=inspected_configurations,found=found)
    
def drop_execute_fn(a, b, s, store):
    
    o = a.args[0]
    g = a.args[1]

    
    at=b.at.copy()
    atRob=b.atRob
    inspected=b.inspected.copy()
    inspected_configurations=b.inspected_configurations.copy()
    found=b.found.copy()
    
    # verify_effects
    if random.random() < DROP_SUCCESS:
        at[o]="pile"
        inspected=list(set(inspected+[o]))
        
    return State(), ObjectObservation(at=at,atRob=atRob,inspected=inspected,inspected_configurations=inspected_configurations,found=found)
    
# closed world assumption: not explicitly True predicates are false

# Set up environment dynamics
class ToyDiscrete(TampuraEnv):
    def initialize(self):
        
        store = AliasStore()
        
        for o in OBJECTS.keys():
            store.set(o,o,"physical")
            
        store.set("robot_gripper",ROBOT["gripper_region"],"region")
        
        # # set robot regions??
        for o in OBJECTS.keys():
            store.set("reg_"+o,OBJECTS[o]["region"],"robot_configuration")
            
        store.set("pile",0,"region") # dumping pile
        # store.certified.append(Atom("is_gripper",["robot_gripper"]))
        store.set("dice","dice","physical_special") # not the best approach
        
                    
        return ObjectsBelief(), store
    

    def get_problem_spec(self) -> ProblemSpec:
        predicates = [ 
            Predicate("at",["physical","region"]),
            Predicate("atRob", ["robot_configuration"]),
            Predicate("grasped",["physical","grasp"]),
            Predicate("inspected",["physical"]),
            Predicate("inspected_configuration", ["robot_configuration"]),
            Predicate("found", ["physical_special"])
        ] 
        
        stream_schemas = [
            
            
            StreamSchema(
                name="sample-robot-location",
                output="?r1",
                output_type="robot_configuration",
                certified=[Atom("atRob",["?r1"])],
                sample_fn=sample_robot_location
                
            ),            

            
            StreamSchema(
                name="sample-grasp",
                inputs=["?o1"],
                input_types=["physical"],
                output="?g1",
                output_type="grasp",
                certified=[Atom("grasped",["?o1","?g1"])], # generates a valid grasp g1 for o1
                sample_fn=sample_grasp                
            ),

        ]
        
        action_schemas = [
            
            ActionSchema(
                name="look",
                inputs=["?q"],
                input_types=["robot_configuration"],
                preconditions=[Atom("atRob",["?q"]),ForAll(Atom("inspected",["?o"]),["?o"],["physical"])], # all known objects have been inspected already
                effects=[Atom("inspected_configuration",["?q"])],
                verify_effects=[Atom("found",["dice"])], 
                execute_fn=look_execute_fn,
                effects_fn=effect_from_execute_fn(look_execute_fn),
            
            ),
            
            
            ActionSchema(
                name="move-robot",
                inputs=["?q"],
                input_types=["robot_configuration"],
                preconditions=[
                               Exists(Atom("atRob",["?r"]),["?r"],["robot_configuration"]), # current location of robot is known
                               Not(Atom("atRob",["?q"])), # robot is not already at region it is moving to
                               ],
                effects=[Atom("atRob",["?q"])], # inspected_region: prevents robot from moving to this location everytime
                execute_fn=move_execute_fn,
                effects_fn=effect_from_execute_fn(move_execute_fn),              
            
            ),

            ActionSchema(
                name="pick",
                inputs=["?o1","?g1"],
                input_types=["physical","grasp"],
                preconditions=[Not(Atom("inspected",["?o1"])), # object has not been inspected yet
                            #    Exists(Atom("at",["?o1","?r1"]),["?r1"],["region"]), # location of ?o1 is known
                               Atom("grasped",["?o1","?g1"]), # ?g1 is a valid grasp for ?o1
                               Not(Exists(Atom("at",["?o","robot_gripper"]),["?o"],["physical"])), # robot gripper is free
                               ], 
                verify_effects=[Atom("at", ["?o1","robot_gripper"])],
                execute_fn=pick_execute_fn,
                effects_fn=effect_from_execute_fn(pick_execute_fn),
            ),
            ActionSchema(
                name="drop",
                inputs=["?o1","?g1"],
                input_types=["physical","grasp"],
                preconditions=[#Not(Atom("inspected",["?o1"])), # object has not been inspected yet
                               Atom("at",["?o1","robot_gripper"]), # robot is holding ?o1
                               Atom("grasped",["?o1","?g1"]), # ?g1 is a valid grasp for ?o1
                            #    Not(Atom("at",["?o1","?r1"])), # current location is not the same as goal location
                               ], 
                verify_effects=[Atom("at",["?o1","pile"]),Atom("inspected",["?o1"]),Not(Atom("at",["?o1","robot_gripper"]))], # verify effects with oneof is not the ideal way
                execute_fn=drop_execute_fn,
                effects_fn=effect_from_execute_fn(drop_execute_fn),
            ),
            NoOp(),
        ]
        
        # reward = Atom("inspected",["o_o1"]) # works for one object
        # reward = And([Atom("inspected",["o_o1"]),Atom("inspected",["o_o2"])]) #symk planning failure: WHY CAN'T IT DEAL WITH TWO OBJECTS??
        reward = Atom("found",["dice"])

        # reward = And([Atom("inspected",[o])for o in OBJECTS.keys()])
        
        spec = ProblemSpec(
            predicates=predicates,
            stream_schemas=stream_schemas,
            action_schemas=action_schemas,
            reward=reward,
        )

        return spec



## Create environment and planner

In [33]:
# Planner
cfg = get_default_config(save_dir=os.getcwd())

# Set some print options to print out abstract belief, action, observation, and reward
cfg["print_options"] = "ab,a,o,r"
cfg["vis_graph"] = True
cfg["flat_sample"] = True
cfg["batch_size"] = 100
cfg["num_samples"] = 1000
cfg["max_steps"] = 20


# Initialize environment
env = ToyDiscrete(config=cfg)
b0, store = env.initialize()

# Set up logger to print info
setup_logger(cfg["save_dir"], logging.INFO)

# Initialize the policy
planner = TampuraPolicy(config = cfg, problem_spec = env.problem_spec)

## Run Planner
Make sure symk is installed (see README) before running the Tampura planner.
With the default settings, the planner should pick both every time.

In [34]:
print(store)
history,store = planner.rollout(env, b0, store)

AliasStore(als={'o_o1': 'o_o1', 'o_o2': 'o_o2', 'robot_gripper': 5, 'reg_o_o1': 1, 'reg_o_o2': 2, 'pile': 0, 'dice': 'dice'}, als_type={'o_o1': 'physical', 'o_o2': 'physical', 'robot_gripper': 'region', 'reg_o_o1': 'robot_configuration', 'reg_o_o2': 'robot_configuration', 'pile': 'region', 'dice': 'physical_special'}, alph_counter={}, certified=[], sample_counts={}, branching_factor={})

==========t=0==========
Abstract Belief: AbstractBelief(items=[])
Reward: 0.0
Flat Stream Sampling
Sampling StreamSchema(name='sample-robot-location', inputs=[], input_types=[], output='?r1', output_type='robot_configuration', preconditions=[], certified=[Atom(pred_name='atRob', args=['?r1'])], sample_fn=<function sample_robot_location at 0x7689f7bf2680>)([])
Sampling StreamSchema(name='sample-grasp', inputs=['?o1'], input_types=['physical'], output='?g1', output_type='grasp', preconditions=[], certified=[Atom(pred_name='grasped', args=['?o1', '?g1'])], sample_fn=<function sample_grasp at 0x7689f7bf3d0

In [35]:
history.actions

[Action(name='pick', args=['o_o2', 'o_gr_1']),
 Action(name='drop', args=['o_o2', 'o_gr_1']),
 Action(name='pick', args=['o_o1', 'o_gr_0']),
 Action(name='drop', args=['o_o1', 'o_gr_0']),
 Action(name='move-robot', args=['reg_o_o2']),
 Action(name='look', args=['reg_o_o2']),
 Action(name='no-op', args=[]),
 Action(name='no-op', args=[]),
 Action(name='no-op', args=[]),
 Action(name='no-op', args=[]),
 Action(name='no-op', args=[]),
 Action(name='no-op', args=[]),
 Action(name='no-op', args=[]),
 Action(name='no-op', args=[]),
 Action(name='no-op', args=[]),
 Action(name='no-op', args=[]),
 Action(name='no-op', args=[]),
 Action(name='no-op', args=[]),
 Action(name='no-op', args=[]),
 Action(name='no-op', args=[]),
 None]

In [30]:
history.abstract_beliefs

[AbstractBelief(items=[]),
 AbstractBelief(items=[Atom(pred_name='at', args=['o_o1', 'robot_gripper'])]),
 AbstractBelief(items=[Atom(pred_name='at', args=['o_o1', 'pile']), Atom(pred_name='inspected', args=['o_o1'])]),
 AbstractBelief(items=[Atom(pred_name='at', args=['o_o1', 'pile']), Atom(pred_name='at', args=['o_o2', 'robot_gripper']), Atom(pred_name='inspected', args=['o_o1'])]),
 AbstractBelief(items=[Atom(pred_name='at', args=['o_o2', 'pile']), Atom(pred_name='at', args=['o_o1', 'pile']), Atom(pred_name='inspected', args=['o_o2']), Atom(pred_name='inspected', args=['o_o1'])]),
 AbstractBelief(items=[Atom(pred_name='at', args=['o_o2', 'pile']), Atom(pred_name='at', args=['o_o1', 'pile']), Atom(pred_name='atRob', args=['reg_o_o1']), Atom(pred_name='inspected', args=['o_o2']), Atom(pred_name='inspected', args=['o_o1'])]),
 AbstractBelief(items=[Atom(pred_name='at', args=['o_o2', 'pile']), Atom(pred_name='at', args=['o_o1', 'pile']), Atom(pred_name='atRob', args=['reg_o_o1']), Atom(

In [151]:
store.certified

[Atom(pred_name='is_gripper', args=['o_re_0']),
 Atom(pred_name='atRob', args=['o_re_1']),
 Atom(pred_name='at', args=['o_o1', 'o_re_2']),
 Atom(pred_name='at', args=['o_o2', 'o_re_3']),
 Atom(pred_name='grasped', args=['o_o1', 'o_gr_0']),
 Atom(pred_name='grasped', args=['o_o2', 'o_gr_1'])]

In [101]:
store.als['o_re_1']

5

In [ ]:
# is_gripper is not set for robot_gripper